In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
data=pd.read_excel('赛题1数据材料\赛题1数据集\一般纳税人.xlsx',index_col='企业编号')

In [3]:
data.index.value_counts()

2638    6
3027    6
2186    6
1648    5
3885    5
1369    5
3886    5
3225    5
3327    4
3216    4
3298    4
3425    4
2730    4
1070    4
3409    4
3888    4
1118    4
1165    4
3309    4
3193    4
3174    4
2866    4
1287    4
3277    4
3086    4
3303    4
3320    4
3175    4
3061    4
1410    4
       ..
2840    1
2846    1
2852    1
2854    1
2726    1
2720    1
2718    1
2716    1
2602    1
2608    1
2612    1
2616    1
2628    1
2632    1
2642    1
2646    1
2658    1
2660    1
2662    1
2666    1
2672    1
2676    1
2690    1
2698    1
2700    1
2702    1
2704    1
2706    1
2710    1
2049    1
Name: 企业编号, Length: 1921, dtype: int64

In [4]:
data_1=pd.get_dummies(data['纳税人资格'],prefix='纳税人资格')

In [5]:
data_1=data_1.groupby('企业编号').sum().T

In [6]:
data_1

企业编号,1001,1002,1004,1005,1006,1008,1009,1010,1011,1012,...,3983,3984,3985,3987,3989,3990,3993,3997,3998,4000
纳税人资格_一般纳税人,0,0,1,1,0,1,0,1,1,0,...,0,0,0,0,1,0,1,0,0,0
纳税人资格_其他,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
纳税人资格_内资企业采购国产设备,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
纳税人资格_出口退（免）税企业,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
纳税人资格_增值税一般纳税人,0,1,0,0,1,0,0,0,0,1,...,0,1,1,0,0,1,0,1,1,1
纳税人资格_增值税小规模纳税人,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
纳税人资格_增值税汇总纳税企业,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
纳税人资格_外商投资企业,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
纳税人资格_外资企业采购国产设备,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
纳税人资格_按征收率征收增值税小规模纳税人,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
data_2=pd.get_dummies(data['登记注册类型'],prefix='登记注册类型')

In [8]:
data_2=data_2.groupby('企业编号').sum().T

In [9]:
data_2

企业编号,1001,1002,1004,1005,1006,1008,1009,1010,1011,1012,...,3983,3984,3985,3987,3989,3990,3993,3997,3998,4000
登记注册类型_中外合资经营企业,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
登记注册类型_其他,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
登记注册类型_其他企业,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
登记注册类型_其他有限责任公司,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
登记注册类型_合资经营企业（港或澳、台资）,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
登记注册类型_国有企业,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
登记注册类型_国有相对控股上市企业,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
登记注册类型_国有相对控股非上市企业,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
登记注册类型_国有绝对控股上市企业,2,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
登记注册类型_国有绝对控股非上市企业,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
data=(data_1).append(data_2)

In [11]:
data

企业编号,1001,1002,1004,1005,1006,1008,1009,1010,1011,1012,...,3983,3984,3985,3987,3989,3990,3993,3997,3998,4000
纳税人资格_一般纳税人,0,0,1,1,0,1,0,1,1,0,...,0,0,0,0,1,0,1,0,0,0
纳税人资格_其他,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
纳税人资格_内资企业采购国产设备,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
纳税人资格_出口退（免）税企业,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
纳税人资格_增值税一般纳税人,0,1,0,0,1,0,0,0,0,1,...,0,1,1,0,0,1,0,1,1,1
纳税人资格_增值税小规模纳税人,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
纳税人资格_增值税汇总纳税企业,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
纳税人资格_外商投资企业,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
纳税人资格_外资企业采购国产设备,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
纳税人资格_按征收率征收增值税小规模纳税人,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
